# Step- 1 : Business Problem Understanding
    
    

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# step : 2 :Data Understanding

load Data & Understand every variable

In [5]:
df = pd.read_excel("insurance.xlsx")
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


Dataset Understanding

In [6]:
df.shape

(1338, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


# Exploratory data analysis

In [12]:
categorical=[]
continous=[]
check=[]

d_types=dict(df.dtypes)

for name,type in d_types.items():
    if str(type) == 'object':
        categorical.append(name)
        
    elif str(type)=="float64":
        continous.append(name)
        
    else:
        check.append(name)
        
print("Categorical feature",categorical)
print("Continous feature",continous)
print("feature to be checked",check)

Categorical feature ['sex', 'smoker', 'region']
Continous feature ['bmi', 'expenses']
feature to be checked ['age', 'children']


In [13]:
d_type=dict(df.dtypes)
for name,type_ in d_types.items():
    if str(type_)=='object':
        print(f"===={name}===>")
        print(df[name].value_counts())

====sex===>
male      676
female    662
Name: sex, dtype: int64
====smoker===>
no     1064
yes     274
Name: smoker, dtype: int64
====region===>
southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64


In [14]:
df.describe()

,age,bmi,children,expenses
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.665471,1.094918,13270.422414
std,14.049960,6.098382,1.205493,12110.011240
min,18.000000,16.000000,0.000000,1121.870000
25%,27.000000,26.300000,0.000000,4740.287500
50%,39.000000,30.400000,1.000000,9382.030000
75%,51.000000,34.700000,2.000000,16639.915000
max,64.000000,53.100000,5.000000,63770.430000


In [15]:
df.corr()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2472\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,age,bmi,children,expenses
age,1.000000,0.109341,0.042469,0.299008
bmi,0.109341,1.000000,0.012645,0.198576
children,0.042469,0.012645,1.000000,0.067998
expenses,0.299008,0.198576,0.067998,1.000000


# step 3 : Data processing

In [16]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [17]:
#drop the region column

df.drop(columns='region',inplace=True)

In [18]:
#Encoding sex columnn

df["sex"].replace({'female':0,'male':1},inplace=True)

#Encoding  smoker column

df['smoker'].replace({'no':0,'yes':1},inplace=True)

# X & Y

In [20]:
x=df.drop(columns='expenses')
y=df['expenses']

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=9)



# step 4 : Modelling And Evaluation

In [24]:
#Modelling

from sklearn.linear_model import ElasticNet

enr_base=ElasticNet()

enr_base.fit(x_train,y_train)

##Predictions
train_predictions=enr_base.predict(x_train)
test_predictions=enr_base.predict(x_test)

#Evaluation

print("Train_R2",enr_base.score(x_train,y_train))
print("Test_R2",enr_base.score(x_test,y_test))

#cross val score

from sklearn.model_selection import cross_val_score

cv=cross_val_score(enr_base,x,y,cv=5).mean()
print(cv)

Train_R2 0.39155822533558726
Test_R2 0.3970277641347352
0.3889250431216655


# Applying Hyper parameter tunning for ElasticNet regression

In [26]:
from sklearn.model_selection import GridSearchCV

#model

estimator = ElasticNet()

#parameters &values

param_grid = {"alpha":[0.1,0.2,1,2,3,5,10],"l1_ratio":[0.1,0.5,0.75,0.9,0.95,1]}


#Identifying the best values of the parameter within given values for the given data

model_hp = GridSearchCV(estimator,param_grid,cv=5,scoring='neg_mean_squared_error')

model_hp.fit(x_train,y_train)

model_hp.best_params_

{'alpha': 10, 'l1_ratio': 1}

# Rebuild ElasticNet Model Using best Hyperparameters

In [33]:
#Modelling 

enr_best = ElasticNet(alpha=10,l1_ratio=1)
enr_best.fit(x_train,y_train)

print("Intersept",enr_best.intercept_)
print("Coefficient",enr_best.coef_)

#predictions

train_predictions= enr_best.predict(x_train)
test_predictions= enr_best.predict(x_test)

#Evaluation

print("Train R2:",enr_best.score(x_train,y_train))
print("test_R2",enr_best.score(x_test,y_test))

#Cross_val_Score

from sklearn.model_selection import cross_val_score

cv= cross_val_score(enr_best,x,y,cv=5).mean()
print(cv)


Intersept -11449.28756082979
Coefficient [ 2.56838444e+02 -6.43158858e-01  3.04860929e+02  4.34656692e+02
  2.35631810e+04]
Train R2: 0.7433083585849637
test_R2 0.7755411716841649
0.7467299170217538


# Predictions on new data

In [34]:
input_data= {"age":31,
            "sex":"female",
            "bmi":25.74,
            "children":0,
            "smoker":"no",
            "region":"northeast"}


# Preprocessing data

In [35]:
df_test = pd.DataFrame(input_data,index=[0])

df_test.drop('region',axis=1,inplace=True)

df_test['sex'].replace({'female':0,'male':1},inplace=True)

df_test['smoker'].replace({'no':0,"yes":1},inplace=True)

transformed_data=df_test

# predict

In [36]:
enr_best.predict(transformed_data)

array([4359.82451623])